# Assemble Program

In [1]:
var x86Assembler = require("../build/x86Assembler/src/assembler")

In [2]:
// Create Assembler object
var assembler = new x86Assembler.Assembler();

In [3]:
// Define code to be compiled
var code = `
    mov $0x100,%esp
    mov $0x1234,%ax
    call L2
L1: jmp L1 ; Infinite Loop
L2: inc %ax
    ret
`

In [4]:
// Assemble this code 
var assembled = assembler.assembleProgram(code);

// Print out the assembled code in tabular manner
console.log(assembled.toString())

Addr			Machine Code			Assembly
--------------------------------------------------------------------------------
0:			bc 00 01 00 00                mov  $0x100,%esp
5:			66 b8 34 12                   mov  $0x1234,%ax
9:			e8 02 00 00 00                call  L2
e:			eb fe                         L1: jmp  L1
10:			66 40                         L2: inc  %ax
12:			c3                            ret  



In [5]:
// Get machine code 
var machineCode = assembled.getMachineCode()
console.log(machineCode)

Uint8Array [
  188,   0,   1,  0,   0, 102, 184,
   52,  18, 232,  2,   0,   0,   0,
  235, 254, 102, 64, 195
]


In [6]:
// Get metadat to be used for Debugging in PC
var metadata = assembled.genMetadata()
console.log(metadata)

{
  program: '\n' +
    '    mov $0x100,%esp\n' +
    '    mov $0x1234,%ax\n' +
    '    call L2\n' +
    'L1: jmp L1 ; Infinite Loop\n' +
    'L2: inc %ax\n' +
    '    ret\n',
  line_nums: [ 2, 3, 4, 5, 6, 7 ],
  symbol_table: { L1: 3, L2: 4 },
  reverse_symbol_table: { '3': 'L1', '4': 'L2' },
  instruction_start_addr: [ 0, 5, 9, 14, 16, 18 ],
  instruction_lengths: [ 5, 4, 5, 2, 2, 1 ],
  addr_instruction_map: { '0': 0, '5': 1, '9': 2, '14': 3, '16': 4, '18': 5 }
}


# Load Code in PC

In [7]:
var PC = require("../build/x86PC/src/pc")

In [23]:
// Create PC with 256 Bytes of memory
var pc = new PC.PC(256)

// Some utility functions
function showIP(pc) {
    console.log("Instruction Pointer : 0x" +pc.cpu.getInstructionPointer().toString(16));
}
function showRegister(pc, reg) {
    console.log("Register "+ reg + " : " +pc.cpu.registers[reg].getHexadecimalBytes().join(" "));
}

function showStack(pc) {
    console.log("Stack: " + pc.memory.getSlice(pc.cpu.registers["esp"].getNumericValue()));
}

In [29]:
// Load the Machine code in PC starting at address 0
pc.loadAssembledProgram(machineCode, 0)
showIP(pc)

Instruction Pointer : 0x0


In [30]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "esp")

mov $0x100, %esp
Instruction Pointer : 0x5
Register esp : 0 0 1 0


In [31]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "eax")

mov $0x1234, %ax
Instruction Pointer : 0x9
Register eax : 0 0 12 34


In [32]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "esp")
showStack(pc)

call 0x10
Instruction Pointer : 0x10
Register esp : 0 0 0 fc
Stack: 14,0,0,0


In [33]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "ax")

inc %ax
Instruction Pointer : 0x12
Register ax : 12 35


In [34]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)
showRegister(pc, "esp")
showStack(pc)

ret 
Instruction Pointer : 0xe
Register esp : 0 0 1 0
Stack: 


In [35]:
// Execute the next instruction
pc.executeNextInstruction()
showIP(pc)

jmp 0xe
Instruction Pointer : 0xe


In [36]:
// Infinite Loop
pc.executeNextInstruction()
showIP(pc)

jmp 0xe
Instruction Pointer : 0xe
